# 📘 Word → Structured JSON/YAML Pipeline
本 Notebook 演示如何将人工整理的 CSV（含 JSON 列）转换为结构化 JSON 文件和 YAML 汇总。

In [ ]:
import pandas as pd, json, yaml, os

# 读取 CSV
df = pd.read_csv('commands.csv')
os.makedirs('data', exist_ok=True)
cmd_objects = []

for _, row in df.iterrows():
    param_json = json.loads(row['参数JSON'])
    cmd_data = {
        'command': row['命令'],
        'title': row['命令标题'],
        'type': [t.strip() for t in row['命令类型'].split(';')],
        'formats': [f.strip() for f in row['命令格式'].split('|')],
        'parameters': param_json,
        'examples': [
            {'cmd': c.strip(), 'resp': r.strip()}
            for c, r in zip(row['示例命令'].split('|'), row['示例响应'].split('|'))
        ],
        'description': row['功能描述'],
        'notes': row['备注']
    }
    cmd_objects.append(cmd_data)

    # 写入单独 JSON 文件
    with open(f"data/{row['命令']}.json", 'w', encoding='utf-8') as f:
        json.dump(cmd_data, f, ensure_ascii=False, indent=2)

# 汇总导出 YAML
with open('all_commands.yaml', 'w', encoding='utf-8') as f:
    yaml.safe_dump({'commands': cmd_objects}, f, allow_unicode=True, sort_keys=False)

print('✅ 已生成 data/*.json 和 all_commands.yaml')